# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$.
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки.
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вершину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии.

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):

    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))

    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер.

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения узлов дерева (используйте, например, `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признаков и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три разных критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score


class DecisionTreeNode:
    def __init__(self, depth, max_depth, min_samples_leaf, max_leaves):
        self.depth = depth
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_leaves = max_leaves
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None
        self.value = None

    def fit(self, X, y):
        unique_classes, class_counts = np.unique(y, return_counts=True)
        self.value = unique_classes[np.argmax(class_counts)]

        if self.depth == self.max_depth\
           or len(X) <= self.min_samples_leaf\
           or self.max_leaves == 0:
            return

        best_info_gain = 0
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                y_left = y[X[:, feature] < threshold]
                y_right = y[X[:, feature] >= threshold]
                info_gain = entropy(y) - (len(y_left) / len(y) * entropy(y_left) + len(y_right) / len(y) * entropy(y_right))

                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    self.feature = feature
                    self.threshold = threshold

        if best_info_gain == 0: return

        X_left, X_right, y_left, y_right = X[X[:, self.feature] < self.threshold], X[X[:, self.feature] >= self.threshold], y[X[:, self.feature] < self.threshold], y[X[:, self.feature] >= self.threshold]

        if self.max_leaves is None:
            self.left = DecisionTreeNode(self.depth + 1, self.max_depth, self.min_samples_leaf, self.max_leaves)
            self.right = DecisionTreeNode(self.depth + 1, self.max_depth, self.min_samples_leaf, self.max_leaves)
        else:
            new_node_max_leaves = self.max_leaves - 1
            new_node_depth = self.depth + 1
            self.left = DecisionTreeNode(new_node_depth, self.max_depth, self.min_samples_leaf, new_node_max_leaves)
            self.right = DecisionTreeNode(new_node_depth, self.max_depth, self.min_samples_leaf, new_node_max_leaves)

        self.left.fit(X_left, y_left)
        self.right.fit(X_right, y_right)

    def predict(self, X):
        if self.feature is None: return np.array([self.value] * len(X))

        predictions = np.zeros(len(X))
        is_left = X[:, self.feature] < self.threshold
        predictions[is_left] = self.left.predict(X[is_left])
        predictions[~is_left] = self.right.predict(X[~is_left])

        return predictions

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


def train_decision_tree(X, y, max_depth=None, min_samples_leaf=1, max_leaves=None):
    tree = DecisionTreeNode(0, max_depth, min_samples_leaf, max_leaves)
    tree.fit(X, y)
    return tree


def evaluate_model(tree, X, y):
    y_pred = tree.predict(X)
    return accuracy_score(y, y_pred)


data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tree_max_depth = train_decision_tree(X_train, y_train, max_depth=5)
tree_min_samples_leaf = train_decision_tree(X_train, y_train, min_samples_leaf=5)
tree_max_leaves = train_decision_tree(X_train, y_train, max_leaves=5)
tree_purity = train_decision_tree(X_train, y_train)


accuracy_max_depth = evaluate_model(tree_max_depth, X_test, y_test)
accuracy_min_samples_leaf = evaluate_model(tree_min_samples_leaf, X_test, y_test)
accuracy_max_leaves = evaluate_model(tree_max_leaves, X_test, y_test)
accuracy_purity = evaluate_model(tree_purity, X_test, y_test)

print("Accuracy (Max Depth):", accuracy_max_depth)
print("Accuracy (Min Samples Leaf):", accuracy_min_samples_leaf)
print("Accuracy (Max Leaves):", accuracy_max_leaves)
print("Accuracy (Purity):", accuracy_purity)

Accuracy (Max Depth): 0.9777777777777777
Accuracy (Min Samples Leaf): 0.9777777777777777
Accuracy (Max Leaves): 0.9777777777777777
Accuracy (Purity): 0.9777777777777777


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес".

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd


data = pd.read_csv("/content/drive/MyDrive/ITMO/5-sem/ml/churn.csv")
data = data.drop("Surname", axis=1)
data = pd.get_dummies(data, prefix=['Geography', 'Gender'], columns=['Geography', 'Gender'])
data.head()

Mounted at /content/drive


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1,15634602,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,2,15647311,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,3,15619304,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,4,15701354,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,5,15737888,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


"""случайная подвыборка бутстрапом"""
def generate_bootstrap_sample(data):
    n = len(data)
    sample_indices = np.random.choice(n, n, replace=True)
    return data.iloc[sample_indices]


def train_random_forest(data, n_estimators, max_depth, min_samples_leaf, max_features):
    forest = []
    for _ in range(n_estimators):
        bootstrap_sample = generate_bootstrap_sample(data)
        X = bootstrap_sample.drop("Exited", axis=1)
        y = bootstrap_sample["Exited"]

        tree = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, max_features=max_features)
        tree.fit(X, y)
        forest.append(tree)

    return forest


def predict_random_forest(forest, data):
    X = data.drop("Exited", axis=1)
    predictions = [tree.predict(X) for tree in forest]
    # голосование большинства
    y_pred = np.mean(predictions, axis=0) >= 0.5
    return y_pred


def feature_importance(forest, data):
    feature_importance_scores = np.zeros(len(data.columns) - 1)
    for tree in forest:
        feature_importance_scores += tree.feature_importances_
    return feature_importance_scores / len(forest)

from sklearn.model_selection import GridSearchCV


param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}


grid_search = GridSearchCV(
    RandomForestClassifier(),
    param_grid=param_grid,
    scoring='accuracy',
    cv=5
)

grid_search.fit(train_data.drop("Exited", axis=1), train_data["Exited"])

print("Best Parameters:", grid_search.best_params_)

# Повторяем обучение с наилучшими параметрами
best_forest = train_random_forest(
    data,
    n_estimators=grid_search.best_params_['n_estimators'],
    max_depth=grid_search.best_params_['max_depth'],
    min_samples_leaf=grid_search.best_params_['min_samples_leaf'],
    max_features=grid_search.best_params_['max_features']
)

Best Parameters: {'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'n_estimators': 200}


In [ ]:
def predict_random_forest(forest, data):
    X = data.drop("Exited", axis=1)
    predictions = [tree.predict(X) for tree in forest]

    y_pred = np.mean(predictions, axis=0) >= 0.5
    return y_pred

y_test = test_data["Exited"]
y_pred_test = predict_random_forest(best_forest, test_data)
accuracy = accuracy_score(y_test, y_pred_test)
print("Best Model Accuracy on Test Set:", accuracy)

Best Model Accuracy on Test Set: 0.9225
